In [3]:
# import geopandas as gpd
# #from shapely.geometry import Point
# import sys
# print(f"Python Version = {sys.version}")
# #print(f"Pandas Version = {pd.__version__}")
# print(f"Geopandas Version = {gpd.__version__}")
# #print(f"rasterio version = {rio._base.gdal_version()}")

Python Version = 3.7.6 | packaged by conda-forge | (default, Jan  7 2020, 21:48:41) [MSC v.1916 64 bit (AMD64)]
Geopandas Version = 0.7.0


# Single Lat Long to GeoTiff CRS

In [1]:
def convert_point(lat,long):
    """Convert Coordinate Reference systems from map lat/long
    to fire raster CRS
    Input: Point of latitude,longitude
    Output: Tuple of (x,y) in CRS coordinates
    """
    
    import geopandas as gpd
    from shapely.geometry import Point
    
    # Declare point
    point = Point(lat,long)
    
    # Set Source CRS (Mapbox or Google Maps)
    src_crs = "EPSG:4326"
    
    # create dataframe from input lat/long
    gdf=gpd.GeoDataFrame(index=[0],crs = src_crs, geometry=[point])
        
    # Change CRS to match Wildfire CRS (3857)
    gdf_tf = gdf.to_crs("epsg:3857")
    
    # pull x and y value out from the POINT attribute, then get
    # the value in the data series at row[0]
    x = gdf_tf.geometry.x.at[0]
    y = gdf_tf.geometry.y.at[0]
        
    return (x,y)

In [2]:
x,y = convert_point(-122.0,36.8)
print(x,y)

-13580977.876779376 4411265.910262686


# List of Fire polygon points in Fire CRS to Lat Long

In [62]:
def convert_polygon(fire_polygon):
    """Convert Coord ref system (CRS) from fire data (EPSG 3857) to map 
    lat/long (EPSG 4326), polygon starts and ends on same point (to close it)
    
    Input: list containing tuples of x,y points in fire CRS that describe a polygon
    Output: list containing tuples of lat/long points that describe the polygon
    """
    from shapely import geometry
    import geopandas as gpd
    import numpy as np
    # create polygon
    poly = geometry.Polygon([(p[0], p[1]) for p in fire_polygon])
    
    #CRS
    src_crs = "EPSG:3857"
    dst_crs = "EPSG:4326"
    
    # Create Geo DataFrame
    gfp = gpd.GeoDataFrame(index=[0],crs=src_crs,geometry=[poly])
    
    # Convert CRS
    gfp2 = gfp.to_crs(dst_crs)
    
    # pull data from dataframe
    polyout = gfp2.iloc[0]['geometry']
    
    # create list of tuples, but longitude is first
    l = list(map(tuple,np.asarray(polyout.exterior.coords)))
    l = list(map(lambda m: (m[1],m[0]), l))
    
    return l
    

In [67]:
points = [(-13580977.876779376, 4411265.910262686),
 (-13348542.780003022, 4411265.910262686),
 (-13348542.780003022, 4172037.896749009),
 (-13580977.876779376, 4172037.896749009),
 (-13580977.876779376, 4411265.910262686)]

In [68]:
x = convert_polygon(points)
print(x)

[(36.79999999999999, -122.0), (36.79999999999999, -119.91200000000002), (35.06000000000001, -119.91200000000002), (35.06000000000001, -122.0), (36.79999999999999, -122.0)]
